In [1]:
import sys
sys.path.insert(0, '/Users/hinzlehome/codeup-data-science/time-series-exercises/')
from utils.imports import *
from utils.acquire import *

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Guidance for the exercise
1. Setup
	- url (base + endpoint)
	- empty list
1. Loop
	1. make a request
	1. handle the response, add to the list
	1. find the next url endpoint
		1. if it's None, stop looping
		1. if it's a string, use it to construct the next url
1. Turn the list into a dataframe

## General Tips

- solve an easy problem first (the items endpoint), then apply that solution to the larger problem (sales)
- informational print statements are helpful as you are developing code, especially inside of a loop to see what changes
- dont' be afraid to command + shift + p (command + shift + c for jupyter lab) "interrupt the kernel"
- curriculum says https://python.zgulde.net, that will work or use https://api.data.codeup.com

In [2]:
items=acquire_items()

cached csv


In [3]:
len(items)

50

In [4]:
def acquire_stores():
	if os.path.exists('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/stores.csv'):
		print('cached csv')
		df = pd.read_csv('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/stores.csv')
		return df
	else:
		domain = 'https://api.data.codeup.com'
		endpoint = '/api/v1/stores'
		stores = []
		x=True
		while x:
			url = domain + endpoint
			response = requests.get(url)
			data = response.json()
			stores.extend(data['payload']['stores'])
			endpoint = data['payload']['next_page']
			if data['payload']['next_page'] == None:
				x=False
				stores=pd.DataFrame(stores)
				stores.to_csv('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/stores.csv', index=False)
				return stores
			else:
				continue

In [5]:
stores=acquire_stores()

cached csv


In [6]:
len(stores)

10

In [7]:
def acquire_sales():
	if os.path.exists('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/sales.csv'):
		print('cached csv')
		df = pd.read_csv('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/sales.csv')
		return df
	else:
		domain = 'https://api.data.codeup.com'
		endpoint = '/api/v1/sales'
		sales = []
		while True:
			url = domain + endpoint
			response = requests.get(url)
			data = response.json()
			sales.extend(data['payload']['sales'])
			endpoint = data['payload']['next_page']
			if endpoint == None:
				sales=pd.DataFrame(sales)
				sales.to_csv('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/sales.csv', index=False)
				return sales
			else:
				continue

In [8]:
sales=acquire_sales()

cached csv


In [9]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   item         913000 non-null  int64  
 1   sale_amount  913000 non-null  float64
 2   sale_date    913000 non-null  object 
 3   sale_id      913000 non-null  int64  
 4   store        913000 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 34.8+ MB


In [21]:
def merge_on_sales():
	stores=acquire_stores()
	items=acquire_items()
	sales=acquire_sales()

	sales = sales.rename(columns={'item': 'item_id', 'store': 'store_id'})
	sales=sales.merge(items,on='item_id')
	sales=sales.merge(stores,on='store_id')

	return sales
	

In [22]:
sales=merge_on_sales()

cached csv
cached csv
cached csv


In [23]:
sales.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [35]:
def de_elec():
	if os.path.exists('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/opsd_germany_daily.csv'):
		de_elec=pd.read_csv('/Users/hinzlehome/codeup-data-science/time-series-exercises/csvs/opsd_germany_daily.csv')
		return de_elec
	else:
		print('csv not found')

In [36]:
df=de_elec()

In [37]:
df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN
